# Part I - (Prosper Loan Data Visualization)
## by (Michael Nwasunja)

## Introduction
> Introduce the dataset

This data set contains 113,937 loans with 81 variables on each loan, including loan amount, borrower rate (or interest rate), current loan status, borrower income, and many others.
Data dictionary to understand the variables more form this link: https://docs.google.com/spreadsheets/d/1gDyi_L4UvIrLTEC6Wri5nbaMmkGmLQBk-Yx3z0XDEtI/edit#gid=0

We are trying to answer these questions

What factors affect a loan’s outcome status?
What affects the borrower’s APR or interest rate?
Are there differences between loans depending on how large the original loan amount was?



## Preliminary Wrangling


In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

> Load in your dataset and describe its properties through the questions below. Try and motivate your exploration goals through this section.

In [20]:
loans = pd.read_csv('prosperLoanData.csv',error_bad_lines=False, engine="python")

Skipping line 11052: unexpected end of data


In [26]:
print(loans.head(5))

                ListingKey  ListingNumber            ListingCreationDate  \
0  1021339766868145413AB3B         193129  2007-08-26 19:09:29.263000000   
1  10273602499503308B223C1        1209647  2014-02-27 08:28:07.900000000   
2  0EE9337825851032864889A          81716  2007-01-05 15:00:47.090000000   
3  0EF5356002482715299901A         658116  2012-10-22 11:02:35.010000000   
4  0F023589499656230C5E3E2         909464  2013-09-14 18:38:39.097000000   

  CreditGrade  Term LoanStatus           ClosedDate  BorrowerAPR  \
0           C    36  Completed  2009-08-14 00:00:00      0.16516   
1         NaN    36    Current                  NaN      0.12016   
2          HR    36  Completed  2009-12-17 00:00:00      0.28269   
3         NaN    36    Current                  NaN      0.12528   
4         NaN    36    Current                  NaN      0.24614   

   BorrowerRate  LenderYield    ...     LP_ServiceFees  LP_CollectionFees  \
0        0.1580       0.1380    ...            -133.18   

In [22]:
print(loans.shape)

(11050, 81)


In [25]:
print(loans.dtypes)

ListingKey                              object
ListingNumber                            int64
ListingCreationDate                     object
CreditGrade                             object
Term                                     int64
LoanStatus                              object
ClosedDate                              object
BorrowerAPR                            float64
BorrowerRate                           float64
LenderYield                            float64
EstimatedEffectiveYield                float64
EstimatedLoss                          float64
EstimatedReturn                        float64
ProsperRating (numeric)                float64
ProsperRating (Alpha)                   object
ProsperScore                           float64
ListingCategory (numeric)                int64
BorrowerState                           object
Occupation                              object
EmploymentStatus                        object
EmploymentStatusDuration               float64
IsBorrowerHom

Out of the 113937 rows and 81 columns, I am going to select only the columns that are related to our exploration of the loan. I will chose about 15 columns to investigate. These columns are related to the amount of loan, dates, rates and other charactrestics about the borrowers.

In [29]:
df_loans = loans.loc[:, ['ListingCreationDate', 'Term', 'LoanStatus', 'ClosedDate', 'BorrowerAPR',
                         'BorrowerRate', 'LenderYield', 'ProsperRating (Alpha)', 'ProsperScore', 'ListingCategory (numeric)',
                         'BorrowerState', 'Occupation', 'EmploymentStatus', 'AmountDelinquent',
                         'IncomeRange', 'StatedMonthlyIncome',
                         'LoanCurrentDaysDelinquent',
                         'LoanOriginalAmount', 'LoanOriginationDate', 
                         'MonthlyLoanPayment', 'LP_ServiceFees', 'Investors']]
df_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11050 entries, 0 to 11049
Data columns (total 22 columns):
ListingCreationDate          11050 non-null object
Term                         11050 non-null int64
LoanStatus                   11050 non-null object
ClosedDate                   5324 non-null object
BorrowerAPR                  11049 non-null float64
BorrowerRate                 11050 non-null float64
LenderYield                  11050 non-null float64
ProsperRating (Alpha)        8264 non-null object
ProsperScore                 8264 non-null float64
ListingCategory (numeric)    11050 non-null int64
BorrowerState                10523 non-null object
Occupation                   10686 non-null object
EmploymentStatus             10826 non-null object
AmountDelinquent             10328 non-null float64
IncomeRange                  11050 non-null object
StatedMonthlyIncome          11050 non-null float64
LoanCurrentDaysDelinquent    11050 non-null int64
LoanOriginalAmount      

In [30]:
df_loans.duplicated().sum()

0

In [31]:
df_loans.isnull().sum()

ListingCreationDate             0
Term                            0
LoanStatus                      0
ClosedDate                   5726
BorrowerAPR                     1
BorrowerRate                    0
LenderYield                     0
ProsperRating (Alpha)        2786
ProsperScore                 2786
ListingCategory (numeric)       0
BorrowerState                 527
Occupation                    364
EmploymentStatus              224
AmountDelinquent              722
IncomeRange                     0
StatedMonthlyIncome             0
LoanCurrentDaysDelinquent       0
LoanOriginalAmount              0
LoanOriginationDate             0
MonthlyLoanPayment              0
LP_ServiceFees                  0
Investors                       0
dtype: int64

In [32]:
df_loans.head()

,ListingCreationDate,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,ProsperRating (Alpha),ProsperScore,ListingCategory (numeric),...,EmploymentStatus,AmountDelinquent,IncomeRange,StatedMonthlyIncome,LoanCurrentDaysDelinquent,LoanOriginalAmount,LoanOriginationDate,MonthlyLoanPayment,LP_ServiceFees,Investors
0,2007-08-26 19:09:29.263000000,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,NaN,NaN,0,...,Self-employed,472.0,"$25,000-49,999",3083.333333,0,9425,2007-09-12 00:00:00,330.43,-133.18,258
1,2014-02-27 08:28:07.900000000,36,Current,NaN,0.12016,0.0920,0.0820,A,7.0,2,...,Employed,0.0,"$50,000-74,999",6125.000000,0,10000,2014-03-03 00:00:00,318.93,0.00,1
2,2007-01-05 15:00:47.090000000,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,NaN,NaN,0,...,Not available,NaN,Not displayed,2083.333333,0,3001,2007-01-17 00:00:00,123.32,-24.20,41
3,2012-10-22 11:02:35.010000000,36,Current,NaN,0.12528,0.0974,0.0874,A,9.0,16,...,Employed,10056.0,"$25,000-49,999",2875.000000,0,10000,2012-11-01 00:00:00,321.45,-108.01,158
4,2013-09-14 18:38:39.097000000,36,Current,NaN,0.24614,0.2085,0.1985,D,4.0,2,...,Employed,0.0,"$100,000+",9583.333333,0,15000,2013-09-20 00:00:00,563.97,-60.27,20


In [33]:
#Adjust Datetime columns
df_loans['ListingCreationDate'] = pd.to_datetime(df_loans['ListingCreationDate']) 
df_loans['ClosedDate'] = pd.to_datetime(df_loans['ClosedDate'])
df_loans['LoanOriginationDate'] = pd.to_datetime(df_loans['LoanOriginationDate'])
df_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11050 entries, 0 to 11049
Data columns (total 22 columns):
ListingCreationDate          11050 non-null datetime64[ns]
Term                         11050 non-null int64
LoanStatus                   11050 non-null object
ClosedDate                   5324 non-null datetime64[ns]
BorrowerAPR                  11049 non-null float64
BorrowerRate                 11050 non-null float64
LenderYield                  11050 non-null float64
ProsperRating (Alpha)        8264 non-null object
ProsperScore                 8264 non-null float64
ListingCategory (numeric)    11050 non-null int64
BorrowerState                10523 non-null object
Occupation                   10686 non-null object
EmploymentStatus             10826 non-null object
AmountDelinquent             10328 non-null float64
IncomeRange                  11050 non-null object
StatedMonthlyIncome          11050 non-null float64
LoanCurrentDaysDelinquent    11050 non-null int64
LoanOrig

In [35]:
# Convert ProsperRating & IncomeRange to ordinal categorical 
ordinal_var_dict = {'ProsperRating (Alpha)': ['N/A','HR','E','D','C', 'B', 'A', 'AA'],
                    'IncomeRange':['$0','$1-24,999','$25,000-49,999','$50,000-74,999','$75,000-99,999','$100,000+']}
for var in ordinal_var_dict:
    ordered_var = pd.api.types.CategoricalDtype(ordered = True, categories = ordinal_var_dict[var])
    df_loans[var] = df_loans[var].astype(ordered_var)

df_loans['ProsperRating (Alpha)'].value_counts()

C      1786
B      1569
A      1424
D      1387
E       934
HR      657
AA      507
N/A       0
Name: ProsperRating (Alpha), dtype: int64

In [36]:
df_loans['IncomeRange'].value_counts()

$25,000-49,999    3102
$50,000-74,999    3090
$100,000+         1674
$75,000-99,999    1622
$1-24,999          691
$0                  57
Name: IncomeRange, dtype: int64

In [37]:
#Adjust datatype for all other categorical columns
df_loans['Occupation'] = df_loans['Occupation'].astype('category')
df_loans['BorrowerState'] = df_loans['BorrowerState'].astype('category')
df_loans['LoanStatus'] = df_loans['LoanStatus'].astype('category')
df_loans['ListingCategory (numeric)'] = df_loans['ListingCategory (numeric)'].astype('category')
df_loans['EmploymentStatus'] = df_loans['EmploymentStatus'].astype('category')

In [38]:
df_loans['Occupation'].value_counts()

Other                                 2741
Professional                          1256
Computer Programmer                    431
Executive                              418
Teacher                                376
Analyst                                355
Administrative Assistant               351
Sales - Commission                     333
Accountant/CPA                         303
Clerical                               302
Skilled Labor                          287
Sales - Retail                         287
Retail Management                      266
Nurse (RN)                             235
Truck Driver                           197
Construction                           183
Engineer - Mechanical                  155
Police Officer/Correction Officer      151
Laborer                                147
Civil Service                          145
Military Enlisted                      120
Food Service                           112
Food Service Management                105
Tradesman -

In [39]:
df_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11050 entries, 0 to 11049
Data columns (total 22 columns):
ListingCreationDate          11050 non-null datetime64[ns]
Term                         11050 non-null int64
LoanStatus                   11050 non-null category
ClosedDate                   5324 non-null datetime64[ns]
BorrowerAPR                  11049 non-null float64
BorrowerRate                 11050 non-null float64
LenderYield                  11050 non-null float64
ProsperRating (Alpha)        8264 non-null category
ProsperScore                 8264 non-null float64
ListingCategory (numeric)    11050 non-null category
BorrowerState                10523 non-null category
Occupation                   10686 non-null category
EmploymentStatus             10826 non-null category
AmountDelinquent             10328 non-null float64
IncomeRange                  10236 non-null category
StatedMonthlyIncome          11050 non-null float64
LoanCurrentDaysDelinquent    11050 non-null

### What is the structure of your dataset?

The dataset contains 113937 rows and 22 columns. Most of the columns are numeric but we also have categorical variables(both ordinal and nominal) and datetime variables.

### What is/are the main feature(s) of interest in your dataset?

I will work with Borrower Rate, Borrower APR, Prosper Score, Credit Score, Original Loan Amount, Monthly Payment, Borrower Occupation, Borrower State and others if needed.

There are some important features to look at including:

    * BorrowerAPR: The Borrower's Annual Percentage Rate (APR) for the loan.
    * ProsperScore: A custom risk score built using historical Prosper data. The score ranges from 1-10, with 10 being the best,or - lowest risk score. Applicable for loans originated after July 2009.
    * LoanOriginationDate: The date the loan was originated.
    * LenderYield: The Lender yield on the loan. Lender yield is equal to the interest rate on the loan less the servicing fee.

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

I think that Loan Status and Employment Status will have a strong impact on loan and the features we are trying to explor. Also the Monthly Income will play a role here and the Term (lenght of the loan) may have an effect. SO I will try to invistigate these more.

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.


> **Rubric Tip**: The project (Parts I alone) should have at least 15 visualizations distributed over univariate, bivariate, and multivariate plots to explore many relationships in the data set.  Use reasoning to justify the flow of the exploration.



>**Rubric Tip**: Use the "Question-Visualization-Observations" framework  throughout the exploration. This framework involves **asking a question from the data, creating a visualization to find answers, and then recording observations after each visualisation.** 




>**Rubric Tip**: Visualizations should depict the data appropriately so that the plots are easily interpretable. You should choose an appropriate plot type, data encodings, and formatting as needed. The formatting may include setting/adding the title, labels, legend, and comments. Also, do not overplot or incorrectly plot ordinal data.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

## Conclusions
>You can write a summary of the main findings and reflect on the steps taken during the data exploration.



> Remove all Tips mentioned above, before you convert this notebook to PDF/HTML


> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML or PDF` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!

